# Experiments with Mondial-Foreign Key using SQLAgent in GPT 3.5

In [1]:

from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from urllib.parse import quote  

from custom_sql_toolkit.custom_sql_database_toolkit import CustomSQLDatabaseToolkit
from custom_sql_toolkit.prompt import (
    CUSTOM_SQL_PREFIX,
    CUSTOM_FORMAT_INSTRUCTIONS,    
)
from langchain.callbacks import get_openai_callback
from langchain.agents import create_sql_agent

from langchain.agents.agent_types import AgentType

from dotenv import load_dotenv
import os
import sys
import json
import time

load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")

from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils



<>:24: SyntaxWarning: invalid escape sequence '\.'
<>:24: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_11116\4113694040.py:24: SyntaxWarning: invalid escape sequence '\.'
  module_path = os.path.join(path,'..\..')


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLAgents\..\..


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_11116\4113694040.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from custom_sql_toolkit.custom_sql_database_toolkit import CustomSQLDatabaseToolkit


# Schema

In [2]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/7_sql_agent_chatgpt_{SCHEMA}_fk.json"

Functions for saving and reading generated queries

In [3]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

## Connection

In [4]:
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db_connection

{'DB_HOST': 'localhost',
 'DB_PORT': '1522',
 'DB_USER_NAME': 'MONDIAL_GPT',
 'DB_PASS': 'TextDB123',
 'DB_NAME': '',
 'SQL_DRIVER': 'oracle+oracledb',
 'SERVICE_NAME': 'xepdb1',
 'SCHEMA': 'MONDIAL_GPT',
 'KEYWORD_SEARCH_API_URL': ''}

### Using SQLDatabase to get all the information from the database

In [5]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)
db.get_table_names()

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

In [6]:
len(db.get_table_names())

46

In [7]:
cusmom_agent_prefix_prompt = CUSTOM_SQL_PREFIX
custom_format_instructions = CUSTOM_FORMAT_INSTRUCTIONS

In [8]:
print(custom_format_instructions)

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the SQL QUERY generated 
Final Answer: the SQL QUERY generated 


## Creating Chain to generate SQL

In [9]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", verbose=True)
toolkit = CustomSQLDatabaseToolkit(db=db.db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    prefix=cusmom_agent_prefix_prompt,
    format_instructions=custom_format_instructions,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_11116\1390492666.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", verbose=True)


In [10]:
with get_openai_callback() as cb:
    sql_query = agent_executor({"input":"What is the area of Thailand?"})
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    print(cb.completion_tokens)
    print(cb.total_cost)
print(sql_query['output'])

C:\Users\Tahsin\AppData\Local\Temp\ipykernel_11116\1712727129.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sql_query = agent_executor({"input":"What is the area of Thailand?"})




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for the area of Thailand are country and countrylocalname.
Action: sql_db_schema
Action Input: 'country, countrylocalname'Error: table_names {"'country", "countrylocalname'"} not found in databaseI made a mistake in the input format for the tables. Let me correct it and try again.
Action: sql_db_schema
Action Input: 'country,countrylocalname'Error: table_names {

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT area FROM country WHERE name = 'Thailand'

> Finished chain.
0
0
0
0.0
SELECT area FROM country WHERE name = 'Thailand'


Total tokens: **3140**<br>
Prompt tokens: **2995**<br>
completion tokens: **145**<br>
Total cost: **0.0047825**<br>

## Preparing natural language queries to run in LLM

In [11]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

# Running queries in LLM to generate SQL

In [12]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 2
count = 0
errors = []
position = 0
for instance in queries:
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(20)
    try:
        with get_openai_callback() as cb:
            start_time = time.time()
            response = agent_executor({"input":instance["question"]})  
            end_time = time.time()
            sql_query = response['output']
            if "Agent stopped" in sql_query:
                errors.append(position)
            else:
                instance["query_string"] = sql_query
                instance['total_tokens'] = cb.total_tokens
                instance['prompt_tokens'] = cb.prompt_tokens
                instance['completion_tokens'] = cb.completion_tokens
                instance['total_cost'] = cb.total_cost  
                instance['time'] = end_time - start_time
                print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
                save_queries(queries)
    except:
        errors.append(position)
    count += 1
    position+=1
queries



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for the area of Thailand are country and countrylocalname.
Action: sql_db_schema
Action Input: 'country, countrylocalname'Error: table_names {"'country", "countrylocalname'"} not found in databaseI made a mistake in the input format for sql_db_schema. I need to remove the single quotes and space between the table names.
Action: sql_db_schema
Action Input: countr

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT name, area FROM country WHERE name = 'Thailand'

> Finished chain.
1 What is the area of Thailand? SELECT name, area FROM country WHERE name = 'Thailand' 3.415135145187378 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for this query are province and population. I should query the schema of these tables.
Action: sql_db_schema
Action Input: province

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT name, area FROM province WHERE area > 10000;

> Finished chain.
2 What are the provinces with an area greater than 10000? SELECT name, area FROM province WHERE area > 10000; 1.4969077110290527 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe table 'language' seems relevant to the question.
Action: sql_db_schema
Action Input: language
CREATE TABLE language (
	country

...

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT inflation FROM economy WHERE country = 'Germany'

> Finished chain.
98 What was Germany's inflation? SELECT inflation FROM economy WHERE country = 'Germany' 1.5525002479553223 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for the type of government in Iran are likely 'country' and 'politics'. I should query the schema of these tables.
Action: sql_

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT government FROM politics WHERE country = 'IRN'

> Finished chain.
99 What type of government is Iran? SELECT government FROM politics WHERE country = 'IRN' 1.65549898147583 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe tables that seem relevant for the percentage of industries in relation to Japan's economy are economy and country. I should query the schema of th

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT name, area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 3.415135145187378},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, area FROM province WHERE area > 10000;',
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 1.4969077110290527},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name \nFROM language \nWHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 2.1729023456573486},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth FROM lake WHERE name = 'Lake Kariba'",
  'type': 'si

In [13]:
errors

[5, 8, 12, 32, 34, 35, 37, 40, 46, 48, 52, 64, 70, 81, 85, 87, 89, 90, 99]

If there are errors

In [14]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
number_of_queries_to_delay = 15
count = 0
queries = read_queries()
for pos in errors:
    instance = queries[pos]
    print(instance['question'])
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(10)
    try:
        with get_openai_callback() as cb:
            response = agent_executor({"input":instance["question"]})  
            sql_query = response['output']
            instance["query_string"] = sql_query
            instance['total_tokens'] = cb.total_tokens
            instance['prompt_tokens'] = cb.prompt_tokens
            instance['completion_tokens'] = cb.completion_tokens
            instance['total_cost'] = cb.total_cost  
            queries[pos] = instance
        save_queries(queries)

        count += 1
    except:
        pass
queries

What is the percentage of religious people are hindu in thailand?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe 'religion' table seems relevant to the question.
Action: sql_db_schema
Action Input: religion
CREATE TABLE religion (
	country VARCHAR(4 CHAR) NOT NULL, 
	name VARCHAR(50 CHAR) NOT NULL, 
	percentage NUMBER, 
	CONSTRAINT religionkey PRIMARY KEY (name, country), 
	CONSTRAINT religion_country_fk FOREIGN KEY(country

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated.

Final Answer: SELECT percentage FROM religion WHERE name = 'Hindu' AND country = 'THA'

> Finished chain.
What are the cities of Cuba?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for cities are likely city and country. I should query the schema for these tables.
Action: sql_db_schema
Action Input: 'city, country'Error: table_names {"'city", "country'"} not found in dat

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


The city names are stored in the city table. I should query the city names from the city table.
Action: custom_sql_db_query
Action Input: SELECT name FROM city ORDER BY nameSELECT name FROM city ORDER BY nameI now know the SQL QUERY generated 
Final Answer: SELECT name FROM city ORDER BY name

> Finished chain.
What is the host province and year of establishment of the organization "International Energy Agency"?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, 

...

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 

Final Answer: SELECT name FROM city WHERE latitude > 45;

> Finished chain.
What is the percentage of industries in relation to Japan's economy?


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe tables that seem relevant for the percentage of industries in relation to Japan's economy are economy and country. I should query the schema of these tables.
Action: sql_db_schema
Action Input: 'eco

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT industry FROM economy WHERE country = 'JPN'

> Finished chain.


[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT name, area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 3.415135145187378},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name, area FROM province WHERE area > 10000;',
  'type': 'simple',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 1.4969077110290527},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name \nFROM language \nWHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 0,
  'prompt_tokens': 0,
  'completion_tokens': 0,
  'total_cost': 0.0,
  'time': 2.1729023456573486},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth FROM lake WHERE name = 'Lake Kariba'",
  'type': 'si

#### Fixing queries

In [15]:

to_fix = [5, 8, 12, 32, 34, 35, 37, 40, 46, 48, 52, 64, 70, 81, 85, 87, 89, 90, 99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    try:
        with get_openai_callback() as cb:
            start_time = time.time()
            response = agent_executor({"input":instance["question"]})  
            end_time = time.time()
            sql_query = response['output']
            if "Agent stopped" in sql_query:
                print("Agent stopped" + str(pos))
            else:
                instance["query_string"] = sql_query
                instance['total_tokens'] = cb.total_tokens
                instance['prompt_tokens'] = cb.prompt_tokens
                instance['completion_tokens'] = cb.completion_tokens
                instance['total_cost'] = cb.total_cost  
                instance['time'] = end_time - start_time
                print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
                q[pos] = instance
                save_queries(q)
    except:
        print("Agent error" + str(pos))



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe 'religion' table seems relevant to the question.
Action: sql_db_schema
Action Input: religion
CREATE TABLE religion (
	country VARCHAR(4 CHAR) NOT NULL, 
	name VARCHAR(50 CHAR) NOT NULL, 
	percentage NUMBER, 
	CONSTRAINT religionkey PRIMARY KEY (name, country), 
	CONSTRAINT religion_country_fk FOREIGN KEY(country) REFERENCES country (code), 
	CONSTRAINT religionpercent CHECK ((

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated.

Final Answer: SELECT percentage FROM religion WHERE name = 'Hindu' AND country = 'THA'

> Finished chain.
6 What is the percentage of religious people are hindu in thailand? SELECT percentage FROM religion WHERE name = 'Hindu' AND country = 'THA' 5.08650016784668 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe relevant tables for cities are likely city, country, and province. I should

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the schema of the relevant tables. I can construct a query to retrieve the cities of Cuba.
Final Answer: 
SELECT name
FROM city
WHERE country = 'CUB'

> Finished chain.
9 What are the cities of Cuba? SELECT name
FROM city
WHERE country = 'CUB' 2.0590012073516846 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe tables that might contain information about organizations are "organization" and "located". I should q

...

Agent error89


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe "city" table seems like the most relevant one to query for cities.
Action: sql_db_schema
Action Input: city
CREATE TABLE city (
	name VARCHAR(50 CHAR) NOT NULL, 
	country VARCHAR(4 CHAR) NOT NULL, 
	province VARCHAR(50 CHAR) NOT NULL, 
	population NUMBER, 
	latitude NUMBER, 
	longitude NUMBER, 
	elevation NUMBER, 
	CONSTRAINT citykey PRIMARY KEY (name, country, pr

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 

Final Answer: SELECT name 
FROM city 
WHERE latitude > 45;

> Finished chain.
91 List all cities that are at a latitude greater than 45 degrees. SELECT name 
FROM city 
WHERE latitude > 45; 1.742499828338623 0.0


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: airport, borders, city, citylocalname, cityothername, citypops, continent, country, countrylocalname, countryothername, countrypops, desert, economy, encompasses, ethnicgroup, geo_desert, geo_estuary, geo_island, geo_lake, geo_mountain, geo_river, geo_sea, geo_source, island, islandin, ismember, lake, lakeonisland, language, located, locatedon, mergeswith, mountain, mountainonisland, organization, politics, population, province, provincelocalname, provinceothername, provpops, religion, river, riveronisland, riverthrough, seaThe tables that seem relevant for the percentage of industries in relation to Japan's economy are economy and country. I should query t

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


I now know the SQL QUERY generated 
Final Answer: SELECT industry FROM economy WHERE country = 'JPN'

> Finished chain.
100 What is the percentage of industries in relation to Japan's economy? SELECT industry FROM economy WHERE country = 'JPN' 3.4965004920959473 0.0
